In [1]:
from vocabulary import Vocabulary
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
from sklearn.metrics import confusion_matrix

import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import warnings

warnings.filterwarnings("ignore")

## 1、特征为residue_vocabulary，其序列残基对应词汇表所做的字符编码

In [4]:
# 将序列残基对应词典转换成特征
data = pd.read_excel('/home/dldx/R-H/code/classification/ML/data/mafft_MSA_R_H.xlsx')

vocabulary = Vocabulary.get_vocabulary_from_sequences(data.Sequence.values)  #将蛋白质序列构造成词典

In [5]:
input_seq_tensor = []
input_lable = []
for i in range(179):
    # 将序列残基进行字符编码
    seq_tensor = vocabulary.seq_to_tensor(data.Sequence[i])
    input_seq_tensor.append(np.array(seq_tensor))
    
input_seq_tensor = np.array(input_seq_tensor)
input_lable = np.array(data.Tier)

In [13]:
np.arange(0, 31, 3)

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30])

In [5]:
# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.42857142857142855
  sens : 66.66666666666666
  spec : 76.19047619047619
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.39756661143299726
  sens : 64.70588235294117
  spec : 73.68421052631578
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.6079600189283705
  sens : 45.0
  spec : 93.75
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.48281334215771954
  sens : 75.0
  spec : 75.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.9166666666666666
  mcc : 0.7756717518813396
  sens : 90.9090909090909
  spec : 92.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.43958889483793556
  sens : 52.63157894736842
  spec : 82.35294117647058
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.30151134457776363
  sens : 71.42857142857143
  spec : 63.63636363636363
数据集划分 random_state：7 时，
Evaluate：
  accuracy 

## 2、特征为residue_disorder，其序列残基对应disorder所做的字符编码

In [7]:
data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')

input_feature = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/all_seq_disorder_R_H.pkl','rb'))
input_lable1 = np.array(data1.Tier)

In [8]:
# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.6181073683361271
  sens : 73.33333333333333
  spec : 85.71428571428571
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.44272445820433437
  sens : 70.58823529411765
  spec : 73.68421052631578
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.8888888888888888
  mcc : 0.8944271909999159
  sens : 80.0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.6625
  sens : 81.25
  spec : 85.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.9166666666666666
  mcc : 0.7756717518813396
  sens : 90.9090909090909
  spec : 92.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.7727139410764462
  sens : 73.68421052631578
  spec : 94.11764705882352
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.6493506493506493
  sens : 78.57142857142857
  spec : 86.36363636363636
数据集划分 random_state：7 时，
Evaluate：
  accuracy :

## 3、特征为21个无规则卷曲的关键区域长度，IDR1, IDR2, IDR3, IDR4, IDR5, IDR6, IDR7, IDR8,IDR9,IDR10,IDR11,IDR12,IDR13,IDR14,IDR15,IDR16,IDR17,IDR18,IDR19,IDR20,IDR21。

In [2]:
data2_R = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_R_1.pkl','rb'))
data2_H = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_H_3.pkl','rb'))

In [21]:
print(data2_R.shape)
print(data2_H.shape)

(72, 21)
(107, 21)


In [5]:
data2_R[65][15]

3

In [6]:
all_21_IDR = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in data2_R]
    H_IDR = [i[position] for i in data2_H]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR添加到一个数组
    all_21_IDR.append(one_IDR)

In [12]:
# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 21个IDR区域分别进行分类
for position in range(21):
    for r in range(0,31):
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[position],input_lable,train_size=0.8,random_state = r)
        # 定义模型
        model = xgb.XGBClassifier()
        # 将模型加入超参数网格搜索和交叉验证     
        model = GridSearchCV(model, param_grid=param_dict, cv=5)
        # 训练数据集
        model = model.fit(Xtrain, Ytrain)
        # 模型评估
        predict = model.predict(Xtest)
        cm = confusion_matrix(Ytest,predict)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        print("数据集划分 random_state：" + str(r) + " 时，")
        print("Evaluate：")
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.472222222222222

数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__2时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.5976143046671969
  sens : 33.33333333333333
  spec : 95.23809523809523
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.27112768548339117
  sens : 41.17647058823529
  spec : 78.94736842105263
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.1122769921845787
  sens : 40.0
  spec : 68.75
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.15780850600989924
  sens : 43.75
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.23417772655640703
  sens : 54.54545454545454
  spec : 72.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.4661384642714042
  sens : 42.10526315789473
  spec

数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.3611111111111111
  mcc : -0.32524997430829383
  sens : 10.0
  spec : 46.15384615384615
数据集划分 random_state：18 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : 0.641688947919748
  sens : 9.090909090909092
  spec : 100.0
数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.30422978640307247
  sens : 7.6923076923076925
  spec : 73.91304347826086
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.8280786712108251
  sens : 8.333333333333332
  spec : 100.0
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.7171371656006362
  sens : 0.0
  spec : 94.44444444444444
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.11869282030941178
  sens : 45.45454545454545
  spec : 68.0
数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.15075567228888181
  sens : 14.285714285714285
  spec : 90.9090909090909
数据集划分 random_state：24 时，
Evaluate：
 

数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.30935921676911454
  sens : 15.0
  spec : 93.75
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.5140603704975992
  sens : 25.0
  spec : 95.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4526019054848144
  sens : 18.181818181818183
  spec : 96.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.7405316311773545
  sens : 26.31578947368421
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.5263613559678152
  sens : 25.0
  spec : 96.42857142857143
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.37113480951260275
  sens : 14.285714285714285
  spec : 95.45454545454545
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.4268947

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__7时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  m

...

数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : -0.06900655593423542
  sens : 12.5
  spec : 82.14285714285714
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__14时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.7367883976130072
  sens : 5.88235294117647
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.6963106238227914
  sens : 15.0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.43481317827315213

数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：24 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：25 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  m

数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：10 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11 时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : -0.75
  sens : 0.0
  spec : 77.77777777777779
数据集划分 random_state：12 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：14 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：15 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：16 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：18 时，
Evaluate：
  acc

数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.6969320524371696
  sens : 5.263157894736842
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.15075567228888181
  sens : 14.285714285714285
  spec : 90.9090909090909
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.17867411451345286
  sens : 12.5
  spec : 64.28571428571429
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.11325777888594835
  sens : 15.789473684210526
  spec : 88.23529411764706
数据集划分 random_state：10 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.7171371656006362
  sens : 5.555555555555555
  spec : 100.0
数据集划分 random_state：12 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens 

数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__21时，evaluation
***********************************************************************


In [7]:
# 重新跑关于IDR16由数据更改的情况

# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[15],input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")
print("以上为IDR__" + str(15 + 1) + "时，evaluation")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.472222222222222

In [7]:
# 将21个区域的无规则卷曲长度一起作为特征
all_R_H_IDR =[]
all_R_H_IDR.extend(data2_R)
all_R_H_IDR.extend(data2_H)

# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR,input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3779644730092272
  sens : 40.0
  spec : 85.71428571428571
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.5907388856996755
  sens : 70.58823529411765
  spec : 84.21052631578947
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.44163005530805016
  sens : 55.00000000000001
  spec : 81.25
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4136725758374611
  sens : 81.25
  spec : 65.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.4651475390503975
  sens : 72.72727272727273
  spec : 80.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.4801528549953749
  sens : 57.89473684210527
  spec : 82.35294117647058
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.5
  mcc : 0.0
  sens : 50.0
  spec : 50.0
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.2040258751821